In [94]:
from __future__ import print_function
import httplib2
import os
import pandas as pd
import sys
import csv
import datetime

from apiclient.discovery import build
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
from oauth2client import tools
from oauth2client.client import flow_from_clientsecrets
from google_auth_oauthlib.flow import InstalledAppFlow

Before running the code below, you should first save the "client_secret.json" and "oauth2_generate.py" file in the same folder of this Notebook.

In [134]:
#After running this code,you will see a new file"youtube_data.py-oauth2" in your folder
% run oauth2_generate.py 


oauth2.py generated


In [125]:
CLIENT_SECRETS_FILE = "client_secret.json"
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'


In [126]:
def get_authenticated_service():
    flow=flow_from_clientsecrets(CLIENT_SECRETS_FILE,scope=SCOPES)
    #Storage("%s-oauth2.json"% sys.argv[0])
    storage=Storage("youtube_data.py-oauth2.json")
    credentials=storage.get()
    if credentials is None or credentials.invalid:
        credentials=tools.run_flow(flow,storage)
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def remove_empty_kwargs(**kwargs):
        good_kwargs = {}
        if kwargs is not None:
            for key, value in kwargs.items():
                if value:
                    good_kwargs[key] = value
        return good_kwargs

def videos_list_most_popular(client, **kwargs):
        kwargs = remove_empty_kwargs(**kwargs)
        response = client.videos().list(**kwargs).execute()
        return response
    
        
def getd(country_list,category_list):   
    region=country_list
    category=category_list
    url='https://www.youtube.com/watch?v='
    t=datetime.datetime.now()
    str(t)
    time=t.strftime("%Y-%m-%d %H:%M")
    client = get_authenticated_service()

    with open ('youtube_trend_data.csv','w',newline='') as fp:
        os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
        work=csv.writer(fp)
        dic={'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music',
                      '15': 'Pets & Animals', '17': 'Sports','19': 'Travel & Events', '20': 'Gaming','22': 'People & Blogs', 
                      '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', 
                      '28': 'Science & Technology', '29': 'Nonprofits & Activism'}

        work.writerow(['title','ChannelTitle','url',
                       'published_At','DataGenerated_At','region_code','sort_category_title','video_category_title','view_Count', 
                       'like_Count', 'dislike_Count', 'favorite_Count', 'comment_Count'])
        if category=={}:
            for re in region:
                Res=videos_list_most_popular(client,
                part='snippet,contentDetails,statistics',
                chart='mostPopular',
                regionCode=re,
                videoCategoryId='',
                maxResults=50)
                for value in Res['items']:
                    work.writerow([value['snippet']['title'],value['snippet']['channelTitle'],
                                   url+value['id'],
                                   value['snippet']['publishedAt'],
                                   time,re,'No_sort',
                                   dic[value['snippet']['categoryId']]]+list(value['statistics'].values())[:])
                else:
                    print('No result')
        else:
            for re in region:
                for cate_id,cate_title in category.items():
                    Res=videos_list_most_popular(client,
                    part='snippet,contentDetails,statistics',
                    chart='mostPopular',
                    regionCode=re,
                    videoCategoryId=cate_id,
                    maxResults=50)
                    for value in Res['items']:
                        work.writerow([value['snippet']['title'],value['snippet']['channelTitle'],
                                       url+value['id'],
                                       value['snippet']['publishedAt'],
                                       time,re,cate_title,
                                       dic[value['snippet']['categoryId']]]+list(value['statistics'].values())[:]) 
        
    print("Imported data successfully from youtube")

Below are all the categories we can get access to 

{'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music',
 '15': 'Pets & Animals', '17': 'Sports','19': 'Travel & Events', '20': 'Gaming','22': 'People & Blogs', 
 '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', 
 '28': 'Science & Technology', '29': 'Nonprofits & Activism'}

Update the categories you want within {} in the cell below

In [127]:
category_list={'2': 'Autos & Vehicles'}

Update the countries you want within[] in the cell below and the country name should be decoded as ISO 3166-1 alpha-2 codes

In [131]:
country_list=['BR']

In [132]:
getd(country_list,category_list)


Imported data successfully from youtube


In [133]:
df=pd.read_csv('youtube_trend_data.csv')
df

,title,ChannelTitle,url,published_At,DataGenerated_At,region_code,sort_category_title,video_category_title,view_Count,like_Count,dislike_Count,favorite_Count,comment_Count
0,TROUXE ELA ESCONDIDO PRA CASA ‹ EduKof ›,AM3NlC,https://www.youtube.com/watch?v=WGwNE6nXOcU,2018-08-20T16:30:01.000Z,2018-08-21 12:38,BR,Autos & Vehicles,People & Blogs,991069,143780,991,0,5199
1,A GARAGEM DOS SONHOS,Renato Garcia,https://www.youtube.com/watch?v=hCNyS91cb2Y,2018-08-20T22:30:01.000Z,2018-08-21 12:38,BR,Autos & Vehicles,Entertainment,972130,147408,1281,0,6711
2,A INTERESSEIRA ME ENCONTROU NO LAGO ( DEU RUIM...,MAIKI021,https://www.youtube.com/watch?v=Pj9u7gFoFtI,2018-08-18T20:00:00.000Z,2018-08-21 12:38,BR,Autos & Vehicles,Entertainment,1095421,133355,5253,0,9002
3,SOFRI UM ACIDENTE,Bruno Correa,https://www.youtube.com/watch?v=-NmQbodROzg,2018-08-20T14:03:54.000Z,2018-08-21 12:38,BR,Autos & Vehicles,Entertainment,414693,44800,3023,0,2391
4,Pagode da Ofensa na Web #80 - No Engarrafamento!,Pagode da Ofensa na Web!,https://www.youtube.com/watch?v=ODkWGVkOnuo,2018-08-17T14:45:01.000Z,2018-08-21 12:38,BR,Autos & Vehicles,Comedy,706193,69471,3253,0,1579
5,ENCONTREI MUITO DINHEIRO DENTRO DE UM BUEIRO D...,IShow,https://www.youtube.com/watch?v=EGDfED2RXak,2018-08-18T22:57:54.000Z,2018-08-21 12:38,BR,Autos & Vehicles,Entertainment,290798,39447,322,0,2050
6,PEGUEI A PANAMERA DO EDU SEM ELE SABER ‹ PORTU...,PortugaPC,https://www.youtube.com/watch?v=v7SBSPKgN7w,2018-08-18T14:00:05.000Z,2018-08-21 12:38,BR,Autos & Vehicles,Entertainment,554831,61460,732,0,2339
7,FICAR RICO.,Nando Moura,https://www.youtube.com/watch?v=WGP7BK-MrWY,2018-08-20T15:34:04.000Z,2018-08-21 12:38,BR,Autos & Vehicles,Education,206947,37804,437,0,2484
8,QUAL É O MELHOR CARRO? [ REZENDE EVIL ],rezendeevil,https://www.youtube.com/watch?v=L91_9otQWtw,2018-08-18T15:00:02.000Z,2018-08-21 12:38,BR,Autos & Vehicles,Entertainment,383688,59975,1192,0,2132
9,ACESSÓRIO NOVO NA XRE DO GRAU 😍,Lucas MotoVlog,https://www.youtube.com/watch?v=cwmJK3r8Yuk,2018-08-20T15:00:05.000Z,2018-08-21 12:38,BR,Autos & Vehicles,People & Blogs,180046,33091,290,0,6632
